In [ ]:
import os
import sys
from quetzal.os.parallel_call import parallel_call_notebook, parallel_call_notebooks

In [ ]:
if not os.path.exists('log/'):
    os.makedirs('log/')

# Launcher

This notebook automatically launches all operations for a complete model run. One can decide to skip network preparation steps (`prep1X` and `prep2X`), as they take multiple hours for the region of Germany and the networks are readily included in the repository.

Detailed explainations of certain steps can be found in the corresponding notebook. All ASSUMPTIONS are gathered in the `input/parameters.xls` file. This file also includes scenarios as columns, which can be run by putting them in the `scenarios` list below.

In [ ]:
kwargs = {'workers':4, 'errout_suffix':True, 'sleep':1,'stdout_path':r'log/out.txt', 'stderr_path':r'log/err.txt'}

In [ ]:
scenarios = ['base'] #['Avoid', 'Shift']

In [ ]:
import time
start = time.time()

In [ ]:
# Should be ran for every scenario to apply socio-demographic developments
parallel_call_notebook('prep10_zones_disagg.ipynb', arg_list=scenarios, **kwargs)

# Network preparation

NOT required for the fist model run (except `prep10`, if you want to update the population or other socio-demographic parameters).

Execute all `prep1` and `prep2` steps. Only neccessary if new network data or assumptions are applied.

In [ ]:
# Network generation
# All output files are already included in input_static
parallel_call_notebooks(
    ('prep11_road.ipynb', scenarios),
    ('prep12_rail_coach.ipynb', scenarios),
    ('prep14_bus.ipynb', scenarios),
    **kwargs)
parallel_call_notebook('prep13_air.ipynb', arg_list=scenarios, **kwargs)

In [ ]:
# Access/egress distances to PT services based on networks and census data
parallel_call_notebook('prep15_census_distances.ipynb', arg_list=scenarios, **kwargs)

In [ ]:
# Original network aggregation and interconnection
# First cluster stops, then aggregate trips
parallel_call_notebook('prep20_cluster.ipynb', arg_list=scenarios, **kwargs)
parallel_call_notebook('prep21_aggregate_pt.ipynb', arg_list=scenarios, **kwargs)
# Run the aggregation twice in order to reduce bus trips properly
parallel_call_notebook('prep21_aggregate_pt.ipynb', arg_list=scenarios, **kwargs)
parallel_call_notebook('prep22_access_egress_road.ipynb', arg_list=scenarios, **kwargs)

# Modelling on a laptop

If only 8GB RAM is available, you can choose between two options to reduce the model size (1 is recommended):
1. If the level of detail should be retained and there is no need to spatial explicitly map energy demands, you can randomly sample the set of origin-destination pairs (i.e. sparsification).
2. Otherwise, the zoning system must be reduced to 401 NUTS3-level zones. This drastically decreases level of detail for short-distance trips, which account for the majority of total trips, but it allows calculation of spatially comprehensive energy demands or emissions.

## Option 1: Sparsify the OD set for a faster, smaller model

Computation of all OD pairs is not required for accurate aggregate results (pkm, choice probabilities, etc.). Hence, the OD set can be sparsified before the pathfinders are executed. The sample size is defined in the general attributes of the `parameters.xls` file. Leave the field empty for no sampling.

IMPORTANT: The `prep30` notebooks search for the `model/de_volumes` file of the `parent` scenario, as defined in the `parameters.xls`, to sparsify the OD set. This ensures that all scenarios, whose results are compared, share the same OD set.

For the very first model run, you thus have to run the following notebooks for the parent scenario (usually `base`) to generate the reference OD set once. Afterwards, you don't have to sparsify again for your derived scenarios. Just run the notebooks for every new scenario from `prep30` onwards, which will rely on the OD set saved in `model/de_volumes` of the parent scenario.

NOTE: For your first run, you must download the `mode_choice_od_composite_cost.csv` file to your `/output/base/` folder from the latest release of quetzal_germany on github.

In [ ]:
# Specify your parent scenario's name:
parent = 'base'
# Calculation of the OD matrix
parallel_call_notebook('model_destination.ipynb', arg_list=[parent], **kwargs)
parallel_call_notebook('model_volumes_endo.ipynb', arg_list=[parent], **kwargs)

## Option 2: Apply the reduced zoning system

Switch to NUTS-level zones and reduce granularity of results.

In [ ]:
parallel_call_notebook('prep10_zones.ipynb', arg_list=scenarios, **kwargs)
# Networks are already computed
# Access/egress distances to PT services based on networks and census data (runs longer)
parallel_call_notebook('prep15_census_distances.ipynb', arg_list=scenarios, **kwargs)
# Network aggregation and inter-connection
# First cluster stops, then aggregate trips
parallel_call_notebook('prep20_cluster.ipynb', arg_list=scenarios, **kwargs)
parallel_call_notebook('prep21_aggregate_pt.ipynb', arg_list=scenarios, **kwargs)
parallel_call_notebook('prep22_access_egress_road.ipynb', arg_list=scenarios, **kwargs)

# Pathfinders and level of service

Required for the first model run. See above, if you need to sparsify the OD set (i.e. if you don't have 200GB of RAM).

If assumptions in `parameters.xls` are changed, the corresponding `prep3X` notebooks should be ran.

Note: All upcoming steps should be run on the same computer, as intermediate model files are pickled and Python's pickle protocols are not stable across different versions.

In [ ]:
# Road
parallel_call_notebook('prep30_pathfinder_road.ipynb', arg_list=scenarios, **kwargs)
parallel_call_notebook('prep31_prices_car.ipynb', arg_list=scenarios, **kwargs)

In [ ]:
# PT
parallel_call_notebook('prep30_pathfinder_pt.ipynb', arg_list=scenarios, **kwargs)
parallel_call_notebook('prep31_prices_pt.ipynb', arg_list=scenarios, **kwargs)
parallel_call_notebook('prep33_non-motor.ipynb', arg_list=scenarios, **kwargs)
# terminates with error from 'assert manual' if there is a saved versions of footpaths
# so that it doesn't have to compute them again

# 4-step modelling

Run these steps in order. Mode choice generates the composite cost, which is an input for generation and distribution. The last step, assignment, also computes total pkm, when inner-zonal traffic is generated before.

In [ ]:
# Mode choice probabilities and composite cost
parallel_call_notebook('model_mode.ipynb', arg_list=scenarios, **kwargs)

In [ ]:
# Generation
# Doesn't need to be run for the base scenario, as generation volumes
# are already saved in the model folder
#parallel_call_notebook('model_generation_exo.ipynb', arg_list=scenarios, **kwargs) # fixed probabilities from MiD2017
parallel_call_notebook('model_generation_endo.ipynb', arg_list=scenarios, **kwargs) # MNL model based on MiD2017

In [ ]:
# Destination choice probabilities
parallel_call_notebook('model_destination.ipynb', arg_list=scenarios, **kwargs)

In [ ]:
# Generation of the OD matrix
# Puts it all together and calculates compulsory trips with doubly constrained method
parallel_call_notebook('model_volumes_endo.ipynb', arg_list=scenarios, **kwargs)
# Alternatively, use volumes from VP2030
# Then, you have to change "volumes_file" parameter in model_assignment manually to "de_volumes_exo"
#parallel_call_notebook('model_volumes_exo.ipynb', arg_list=scenarios, **kwargs)

In [ ]:
# Generation of inner-zonal volumes, pkm, prices, and times
parallel_call_notebook('model_inner-zonal.ipynb', arg_list=scenarios, **kwargs)

In [ ]:
# Assignment
parallel_call_notebook('model_assignment.ipynb', arg_list=scenarios, **kwargs)

# Equilibration

Demand-supply equilibration happens iteratively for road traffic. It can be used to refine results after running the assignment step once. If so, a new model folder `<scenario name>_equilibrium` is created from which the modelling steps take their road LoS table and save the equilibrated results.

NOTE: Currently inactive due to missing link capacity data.

In [ ]:
# Run the equilibrium road pathfinder to create the new LoS table
parallel_call_notebook('model_assignment_equilibrium.ipynb', arg_list=scenarios, **kwargs)
parallel_call_notebook('prep31_prices_car.ipynb', arg_list=scenarios, **kwargs)
# Rename the scenarios
scenarios = [str(s)+'_equilibrium' for s in scenarios]

In [ ]:
# Demand model
parallel_call_notebook('model_mode.ipynb', arg_list=scenarios, **kwargs)
parallel_call_notebook('model_generation_endo.ipynb', arg_list=scenarios, **kwargs)
parallel_call_notebook('model_destination.ipynb', arg_list=scenarios, **kwargs)
parallel_call_notebook('model_volumes_endo.ipynb', arg_list=scenarios, **kwargs)
# Supply model
parallel_call_notebook('model_assignment.ipynb', arg_list=scenarios, **kwargs)

# Post-processing

Calculate total transport system indicators such as pkm, energy demand, or GHG emissions.

Other validation notebooks have the prefix `val`.

In [ ]:
# Calculation of emissions and total pkm
parallel_call_notebook('post_emissions.ipynb', arg_list=scenarios, **kwargs)

In [ ]:
# Total execution time (minutes)
int((time.time() - start) / 60)